# Generate EMA using daily data service - F#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *exponential moving average - EMA* from a daily data service.

### Inputs/outputs
Exponential moving averages' requests require instrument's identifier, date time intervals and number of points as per inputs.

EMA reduces the lag by applying greater weight to recent data points.The weighting applied to the most recent price depends on the number of points in the moving average The smaller the point count is, the more weight is applied to the most recent price.

N-points EMA data point is computed using the previous' *EMA* value and the wighting multiplier as follows:

$$ EMA = (Close-EMAp)*k+EMAp $$

Where:
* *Price* is the close price
* *EMAp* is the previous points' *EMA* value, *SMA* is used as the EMAp in the first calculation
* *N* is the count of points
* *$ k = \frac{2}{N+1} $* is the *Weighting Multiplier*

<div class="alert alert-block alert-info">
    <b>Note:</b> <i>EMA</i> adapts quickly to price changes than the <i>SMA</i>, when a price reverses direction, the <i>EMA</i> will reverse direction quicker than the <i>SMA</i> due to the weighting 
</div>

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve EMA indicator from the hosted service. The queried endpoint in this script are:
* *DailyEmaSerice*: to directly retrieve daily EMA data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily EMA sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.9.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.DailyAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data
To request *daily ema* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The EMA parameters

#### 3.1 Instrument selection

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 EMA parameters

In [ ]:
// Set EMAs lengths
let longEma = 200
let shortEma = 50

#### 3.3 Time period selection

In [ ]:
// Set data interval (we are using Google date time format)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2019, Month = 01, Day = 01 ),
    EndDate = new Date ( Year = 2020, Month = 12, Day = 31 )
  )

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)

#### 3.4 EMAs request creation
The following code snippet creates *gRPC client*, process daily EMA requests and returns the reply.

In [ ]:
// Instantiate the daily Ema service
let service = new DailyEmaService.DailyEmaServiceClient(channel)

In [ ]:
// Create and process the daily long Ema request
let dailyLongEmaRequest = 
    new DailyEmaRequest( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker ), 
        Length = longEma,
        Constraints = constraints
    )
let dailyLongEmaReply = service.DailyEma(dailyLongEmaRequest, headers)

// Create and process the daily short Ema request
let dailyShortEmaRequest = 
    new DailyEmaRequest ( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker ), 
        Length = shortEma,
        Constraints = constraints
    )
let dailyShortEmaReply = service.DailyEma(dailyShortEmaRequest, headers)

// Visualize requests' results
//dailyLongEmaReply.Data |> display
//dailyShortEmaReply.Data |> display

### Step 4: Visualize data

#### 4.1 Select data

In [ ]:
// Extract the price and the EMAs
let prices       = (dailyLongEmaReply.Data  |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Value)))
let longEmaData  = (dailyLongEmaReply.Data  |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Average)))
let shortEmaData = (dailyShortEmaReply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Average)))

#### 4.2 Plot EMAs and daily prices
Plot EMA request results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:
[
    new Scattergl(
        name = $"{longEma}-days EMA",
        mode = "lines",
        x = (longEmaData |> Seq.map fst),
        y = (longEmaData |> Seq.map snd)
    )
    new Scattergl(
        name = $"{shortEma}-days EMA",
        mode = "lines",
        x = (shortEmaData |> Seq.map fst),
        y = (shortEmaData |> Seq.map snd)
    )
    new Scattergl(
        name = "Price",
        mode = "lines",
        x = (prices |> Seq.map fst),
        y = (prices |> Seq.map snd)
    )
]
|> Chart.Plot
|> Chart.WithTitle $"Long & short EMAs and daily prices for {ticker}"
|> Chart.WithXTitle "Date"
|> Chart.WithWidth  1500
|> Chart.WithHeight 800
|> display